In [1]:
import dash
import json
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import plotly.express as px
import xlrd
import dash_bootstrap_components as dbc
from plotly.subplots import make_subplots
py.init_notebook_mode(connected=True)

In [2]:
# abre o arquivo
f = open("Anuário Estatístico de Energia Elétrica 2020 - Workbook.xlsx - Tabela 2.4.csv",encoding="utf8")
# f.read lê todo o conteúdo do arquivo e retorna uma string
# .split("\n") divide essa string do conteudo em linhas
content = f.read().split("\n")
#lista vazia que armazena os anos
years = []
#for lê a oitava linha que contem os anos dentro do arquivo csv
#.split(",") separa elementos por vírgula da lista, começando pelo elemento 2 e excluindo os últimos 4
for y in content[8].split(",")[2:-4]:
    #insere na lista years cada elemento de y transformado de string para inteiro
    years.append (int(y))

def filtra_dados(regiao):
    data = []
    #for lê todas as linhas da lista
    for l in content:
        #.split(",") separa por vírgula
        ls = l.split(",")
        #condiciona que se o elemento 1 da lista for igual à "Norte" prossigo
        if ls[1] == regiao:
            #determina o começo e fim da lista, partindo do elemento 2 e excluindo os últimos 5
            dt = ls[2:-5]
            #lê cada elemento da lista filtrada
            for l in dt:
                #insere na lista data cada elemento l transformado de string para float
                data.append(float(l))
    return data



regioes = ['Norte', 'Nordeste', 'Centro-Oeste', 'Sul', 'Sudeste']#lista das regiões



In [3]:
# Leitura do arquivo.csv
df = pd.read_csv('consumo.csv', encoding='UTF-8', sep=';') # Ler o arquivo em csv, o UTF-8 Ler os acentos cedilhas
dados=df.values  # Transformação do dataframe em array

# Criação das listas vazias
dados_2012=[]
dados_2013=[]
dados_2014=[]
dados_2015=[]
dados_2016=[]
dados_2017=[]
dados_2018=[]
regioes=[]
colors=['#07325a','#135090','#2178bb','#6ca1cf','#91b0d8'] # Atribuição das cores

# Filtragem de dados em listas usando o laço for
for dado in dados:
    # Adicionando os elementos de acordo com a posição
    regioes.append(dado[0]) 
    dados_2012.append(dado[1])
    dados_2013.append(dado[2])
    dados_2014.append(dado[3])
    dados_2015.append(dado[4])
    dados_2016.append(dado[5])
    dados_2017.append(dado[6])
    dados_2018.append(dado[7])

# exclusão dos elementos desnecessários 
del regioes[0:6]  
del dados_2012[0:6]
del dados_2013[0:6]
del dados_2014[0:6]
del dados_2015[0:6]
del dados_2016[0:6]
del dados_2017[0:6]
del dados_2018[0:6]

# Criação do gráfico Sunburst
# Criação da lista para atribuição dos nomes que aparecem no gráfico 
labels=['REGIÕES']+ regioes+['2012']*5+['2013']*5+['2014']*5+['2015']*5+['2016']*5+['2017']*5+['2018']*5
# Atribuição de quais elementos da lista label são filhos de quem
parents=['']+['REGIÕES']*5+regioes*7 
# Atribuição dos valores referente a lista label
values=['']+[0]*5+dados_2012 +dados_2013 +dados_2014+dados_2015+dados_2016+dados_2017+dados_2018
'''print (labels)
print (parents)
print (values)'''

# Atribuição dos elementos que vão aparecer no gráfico
consumo_livre =go.Figure(go.Sunburst(labels=labels,parents=parents,values=values))
consumo_livre.update_traces(hoverinfo="label+value+percent parent") # Informações do hover
consumo_livre.update_layout(title=dict(    #"dict()"Função que atribui uma série de caracteristicas a variável(dicionário)
    text='Consumo Livre por Região [GWh]',
    font=dict(size=20),
    xref='paper', # Área central do gráfico
    yref='container', # Área externa ao paper
    x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
    y=0.95 # Faz o posicionamento vertical do texto de acordo com yref
),
height=700, # Condicionamento do tamanho do texto
sunburstcolorway =colors,
extendsunburstcolors = True) # Concede aos filhos do elemento pai uma variação da mesma cor

In [4]:
#---------------------------------------------Buscando arquivo.xls e a tabela-----------------------------------------------
wb= xlrd.open_workbook('base_consumo.xls')             #xlrd Abre o arquivo xls
wc= xlrd.open_workbook('base_pibcorrente.xls')        #xlrd Abre o arquivo xls
p= wb.sheet_by_name('Tabela 3.1')                      #sheet Escolhe a tabela a partir do nome
p1=wc.sheet_by_name('Tabela')                          #sheet Escolhe a tabela a partir do nome

#Pegando apenas os dados da tabela que nos interessa de consumo
dados=[]                                               #Declara uma matriz
coluna=[] 
for i in range(7):                                     #Executa o ciclo criando uma lista de 0 a 7
    j=i+2                                              #Receberá os valores das colunas das tabelas e define o que acontece depois de receber um número 
    coluna=p.col_values(j)                             #Recebe os valores das tabelas
    dados.append(coluna[10:15])                        #Pega os valores da coluna e transforma em um dado na matriz

#Pegando apenas os dados da tabela de PIB
i=0                                                    #Declara uma variável como vazio
j=0                                                    #Declara uma variável como vazio
dados1=[]                                              #Declara uma matriz
coluna1=[]                                              
for i in range(7):                                     #Executa o ciclo criando uma lista de 0 a 7
    j=i+1                                              #Receberá os valores das colunas das tabelas e define o que acontece depois de receber um número 
    coluna1=p1.col_values(j)                           #Recebe os valores das tabelas
    dados1.append(coluna1[4:9])                        #Pega os valores da coluna e transforma em um dado na matriz

#___________________________________________________________________________________________________________________________
# ----------------------------------------------------GRÁFICOS--------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------NORTE----------------------------------------------------------------
#Filtrar dados de Consumo
h=0                                                    #Declara vairável vazia
tabela = [[1 for i in range(3)] for i in range(7)]     #Declaro a matrix tabela 3x7
for q in range (7):                                    #Variável armazena os dados da região, determinando q receba e finalize em 7
    for l in range (1):                                #Formando a matriz, determinando q l receba dado e siga as instruções seguintes
        tabela[h][0]= dados[q][l]                        
        tabela[h][1]= (2012+q)                         
        tabela[h][2]= "norte"
        h=h+1   
        
anos= []
for dado in tabela:                                    #Procura o dado na tabela
    anos.append(dado[1])                               #Adiciona os valores dos dados aos anos
valor= []         
for dado in tabela:                                    #Procura o dado na tabela
    valor.append(dado[0])                              #Adiciona os valores dos dados de consumo

    
#Filtar dados de PIB
h=0                                                    #Declara vairável vazia
tabela1 = [[1 for i in range(3)] for i in range(7)]    #Declaro a matrix tabela 3x7
for q in range (7):                                    #Variável armazena os dados da região, determinando q receba e finalize em 7
    for l in range (1):                                #Formando a matriz, determinando q l receba dado e siga as instruções seguintes
        tabela1[h][0]= dados1[q][l]
        tabela1[h][1]= (2012+q)
        tabela1[h][2]= "norte"
        h=h+1  
        
anos1= []
for dadopib in tabela1:                                #Procura o dado na tabela
    anos1.append(dadopib[1])                           #Adiciona os valores dos dados aos anos
valor1= []
for dadopib in tabela1:                                #Procura o dado na tabela
    valor1.append(dadopib[0])                          #Adiciona os valores dos dados ao pib 

    
    
# Criar grafico com dois eixos y
#Para criar gráfico com dois eixos, é necessário implementar do Boolean para make_subplots e eixos

fig = make_subplots(specs=[[{"secondary_y": True}]])   #Cria a fig com dois eixos y

# Add traces/linhas
fig.add_trace(
    go.Scatter(x=anos, y=valor, name="consumo",line = {'color': '#07325a'}),
    secondary_y=False,                                 #Determina uma linha como False para a fig de dois y
)

fig.add_trace(
    go.Scatter(x=anos, y=valor1, name="PIB",line = {'color': '#2178bb'}),
    secondary_y=True,                                  #Determina uma linha como False para a fig de dois y
)

# Add figure title/título
fig.update_layout(title = dict(
                                text='Consumo e PIB na Região Norte [MWh/R$]',
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95),
)

# Set x-axis title
fig.update_xaxes(title_text="Anos")                    #Nomeia linha x como anos

# Set y-axes titles
fig.update_yaxes(title_text="Consumo", secondary_y=False)
fig.update_yaxes(title_text="PIB", secondary_y=True)
fig.update_layout(plot_bgcolor="#e8ecf4")              #Background grafico
#fig.update_layout(paper_bgcolor="#A9E0ED")            #Background grafico
fig.update_layout(font_color="black")                  #Cor da legenda 

#Plota gráfico
fig.show()                                                 

#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------NORDESTE----------------------------------------------------------------

#filtrar dados de Consumo
h=0                                                    #Declara vairável vazia
tabela2 = [[1 for i in range(3)] for i in range(7)]    #Declaro a matriz tabela 3x7
for q in range (7):                                    #Variável armazena os dados da região, determinando q receba e finalize em 7
    for l in range (1):                                #Formando a matriz, determinando q l receba dado e siga as instruções seguintes
        tabela2[h][0]= dados[q][l+1]
        tabela2[h][1]= (2012+q)
        tabela2[h][2]= "nordeste"
        h=h+1  
        
anos2= []
for dado2 in tabela2:                                  #Procura o dado na tabela
    anos2.append(dado2[1])                             #Adiciona os valores dos dados aos anos
valor2= []
for dado2 in tabela2:                                  #Procura o dado na tabela
    valor2.append(dado2[0])                            #Adiciona os valores dos dados de consumo

#filtar dados de PIB
h=0                                                    #Declara vairável vazia
tabela3 = [[1 for i in range(3)] for i in range(7)]    #Declaro a matrix tabela 3x7
for q in range (7):                                    #Variável armazena os dados da região, determinando q receba e finalize em 7
    for l in range (1):                                #Formando a matriz, determinando q l receba dado e siga as instruções seguintes
        tabela3[h][0]= dados1[q][l+1]
        tabela3[h][1]= (2012+q)
        tabela3[h][2]= "nordeste"
        h=h+1        
anos3= []
for dado3 in tabela3:                                  #Procura o dado na tabela
    anos3.append(dado3[1])                             #Adiciona os valores dos dados aos anos
valor3= []
for dado3 in tabela3:                                  #Procura o dado na tabela
    valor3.append(dado3[0])                            #Adiciona os valores dos dados ao pib 

#Criar grafico com dois eixos y
#Para criar gráfico com dois eixos, é necessário implementação do Boolean para make_subplots e eixos
 
fig1 = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces/linhas
fig1.add_trace(
    go.Scatter(x=anos, y=valor2, name="consumo",line = {'color': '#07325a'}),
    secondary_y=False,                                 #Determina uma linha como False para a fig de dois y
)

fig1.add_trace(
    go.Scatter(x=anos, y=valor3, name="PIB",line = {'color': '#2178bb'}),
    secondary_y=True,                                  #Determina uma linha como True para a fig de dois y
)

fig1.update_layout(title = dict(
                                text='Consumo e PIB na Região Nordeste [MWh/R$]',
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95),
)

# Set x-axis title
fig1.update_xaxes(title_text="Anos")#Nomeia linha x como anos

# Set y-axes titles
fig1.update_yaxes(title_text="Consumo", secondary_y=False)
fig1.update_yaxes(title_text="PIB", secondary_y=True)
fig1.update_layout(plot_bgcolor="#e8ecf4")             #Background grafico
#fig1.update_layout(paper_bgcolor="#A9E0ED")           #Background grafico
fig1.update_layout(font_color="black")                 #Cor da legenda 

#Plota gráfico
fig1.show()

#---------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------SUDESTE----------------------------------------------------------------

#Filtrar dados de Consumo
h=0                                                   #Declara vairável vazia
tabela4 = [[1 for i in range(3)] for i in range(7)]   #Declaro a matrix tabela 3x7
for q in range (7):                                   #Variável armazena os dados da região, determinando q receba e finalize em 7
    for l in range (1):                               #Formando a matriz, determinando q l receba dado e siga as instruções seguintes
        tabela4[h][0]= dados[q][l+2]
        tabela4[h][1]= (2012+q)
        tabela4[h][2]= "Sudeste"
        h=h+1        
valor4= []
for dado4 in tabela4:                                 #Procura o dado na tabela
    valor4.append(dado4[0])                           #Adiciona os valores dos dados de consumo

#filtar dados de PIB
h=0                                                   #Declara vairável vazia
tabela5 = [[1 for i in range(3)] for i in range(7)]   #Declaro a matrix tabela 3x7
for q in range (7):                                   #Variável armazena os dados da região, determinando q receba e finalize em 7
    for l in range (1):                               #Formando a matriz, determinando q l receba dado e siga as instruções seguintes
        tabela5[h][0]= dados1[q][l+2]
        tabela5[h][1]= (2012+q)
        tabela5[h][2]= "Sudeste"
        h=h+1        
valor5= []
for dado5 in tabela5:                                 #Procura o dado na tabela
    valor5.append(dado5[0])                           #Adiciona os valores dos dados de PIB

#Criar grafico com dois eixos y
#Para criar gráfico com dois eixos, é necessário implementação do Boolean para make_subplots e eixos

fig2 = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces/linhas
fig2.add_trace(
    go.Scatter(x=anos, y=valor4, name="consumo",line = {'color': '#07325a'}),
    secondary_y=False,                                #Determina uma linha como False para a fig de dois y
)

fig2.add_trace(
    go.Scatter(x=anos, y=valor5, name="PIB",line = {'color': '#2178bb'}),
    secondary_y=True,                                 #Determina uma linha como True para a fig de dois y
)

# Add figure title
fig2.update_layout(title = dict(
                                text='Consumo e PIB na Região Sudeste [MWh/R$]',
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95),
)

# Set x-axis title
fig2.update_xaxes(title_text="Anos")                  #Nomeia linha x como anos

# Set y-axes titles
fig2.update_yaxes(title_text="Consumo", secondary_y=False)
fig2.update_yaxes(title_text="PIB", secondary_y=True)
fig2.update_layout(plot_bgcolor="#e8ecf4")            #Background grafico
#fig2.update_layout(paper_bgcolor="#A9E0ED")          #Background grafico
fig2.update_layout(font_color="black")                #Cor da legenda 

#Plota gráfico
fig2.show()

#--------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------Sul-------------------------------------------------------------

#filtrar dados de Consumo
h=0                                                   #Declara vairável vazia
tabela6 = [[1 for i in range(3)] for i in range(7)]   #Declaro a matrix tabela 3x7
for q in range (7):                                   #Variável armazena os dados da região, determinando q receba e finalize em 7
    for l in range (1):                               #Formando a matriz, determinando q l receba dado e siga as instruções seguintes
        tabela6[h][0]= dados[q][l+3]
        tabela6[h][1]= (2012+q)
        tabela6[h][2]= "Sul"
        h=h+1        
valor6= []
for dado6 in tabela6:                                 #Procura o dado na tabela
    valor6.append(dado6[0])                           #Adiciona os valores dos dados de consumo

#filtar dados de PIB
h=0                                                   #Declara vairável vazia
tabela7 = [[1 for i in range(3)] for i in range(7)]   #Declaro a matrix tabela 3x7
for q in range (7):                                   #Variável armazena os dados da região, determinando q receba e finalize em 7
    for l in range (1):                               #Formando a matriz, determinando q l receba dado e siga as instruções seguintes
        tabela7[h][0]= dados1[q][l+3]
        tabela7[h][1]= (2012+q)
        tabela7[h][2]= "sul"
        h=h+1        
valor7= []
for dado7 in tabela7:                                 #Procura o dado na tabela
    valor7.append(dado7[0])                           #Adiciona os valores dos dados de PIB 

#Criar grafico com dois eixos y
#Para criar gráfico com dois eixos, é necessário implementação do Boolean para make_subplots e eixos

fig3 = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces/linhas
fig3.add_trace(
    go.Scatter(x=anos, y=valor6, name="consumo",line = {'color': '#07325a'}),
    secondary_y=False,                                #Determina uma linha como False para a fig de dois y
)

fig3.add_trace(
    go.Scatter(x=anos, y=valor7, name="PIB",line = {'color': '#2178bb'}),
    secondary_y=True,                                 #Determina uma linha como True para a fig de dois y
)

fig3.update_layout(title = dict(
                                text='Consumo e PIB na Região Sul [MWh/R$]',
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95),
)

# Set x-axis title
fig3.update_xaxes(title_text="Anos")                   #Nomeia linha x como anos

# Set y-axes titles
fig3.update_yaxes(title_text="Consumo", secondary_y=False)
fig3.update_yaxes(title_text="PIB", secondary_y=True)
fig3.update_layout(plot_bgcolor="#e8ecf4")             #Background grafico
#fig3.update_layout(paper_bgcolor="#A9E0ED")           #Background grafico
fig3.update_layout(font_color="black")                 #Cor da legenda 

#Plota gráfico
fig3.show()
#--------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------CENTRO-OESTE----------------------------------------------------

#filtrar dados de Consumo
h=0                                                    #Declara vairável vazia
tabela8 = [[1 for i in range(3)] for i in range(7)]    #Declaro a matrix tabela 3x7
for q in range (7):                                    #Variável armazena os dados da região, determinando q receba e finalize em 7
    for l in range (1):                                #Formando a matriz, determinando q l receba dado e siga as instruções seguintes
        tabela8[h][0]= dados[q][l+4]
        tabela8[h][1]= (2012+q)
        tabela8[h][2]= "Centro-Oeste"
        h=h+1        
valor8= []
for dado8 in tabela8:                                  #Procura o dado na tabela
    valor8.append(dado8[0])                            #Adiciona os valores dos dados de consumo

#filtar dados de PIB
h=0                                                    #Declara vairável vazia
tabela9 = [[1 for i in range(3)] for i in range(7)]    #Declaro a matrix tabela 3x7
for q in range (7):                                    #Variável armazena os dados da região, determinando q receba e finalize em 7
    for l in range (1):                                #Formando a matriz, determinando q l receba dado e siga as instruções seguintes
        tabela9[h][0]= dados1[q][l+4]
        tabela9[h][1]= (2012+q)
        tabela9[h][2]= "Centro-Oeste"
        h=h+1        
valor9= []
for dado9 in tabela9:                                 #Procura o dado na tabela
    valor9.append(dado9[0])                           #Adiciona os valores dos dados de PIB

#Criar grafico com dois eixos y
#Para criar gráfico com dois eixos, é necessário implementação do Boolean para make_subplots e eixos

fig4 = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces/linhas
fig4.add_trace(
    go.Scatter(x=anos, y=valor8, name="consumo",line = {'color': '#07325a'}),
    secondary_y=False,                                #Determina uma linha como False para a fig de dois y
)

fig4.add_trace(
    go.Scatter(x=anos, y=valor9, name="PIB",line = {'color': '#2178bb'}),
    secondary_y=True,                                 #Determina uma linha como True para a fig de dois y
)

fig4.update_layout(title = dict(
                                text='Consumo e PIB na Região Centro-Oeste [MWh/R$]',
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95),
)

# Set x-axis title
fig4.update_xaxes(title_text="Anos")                  #Nomeia linha x como anos

# Set y-axes titles
fig4.update_yaxes(title_text="Consumo", secondary_y=False)
fig4.update_yaxes(title_text="PIB", secondary_y=True)
fig4.update_layout(plot_bgcolor="#e8ecf4")            #Background grafico
#fig4.update_layout(paper_bgcolor="#A9E0ED")          #Background grafico
fig4.update_layout(font_color="black")                #Cor da legenda  

#Plota gráfico
fig4.show()


In [5]:
# Acessa o  arquivo

df = xlrd.open_workbook('202.xls')
tabela = df.sheet_by_name('Tabela 2.14')

# 2012
coluna = []
dados = []
x = '2012'
for i in range(8):
    for x in range(2):
        j = x + 1
        coluna = tabela.col_values(j)
        dados.append(coluna[10:15])
#print(dados)

# Relação com o ano 2012

h = 0
m = 0
tabela = [[1 for x in range(2)]for x in range(5)]
for q in range(5):
    for l in range(1):
        tabela[h][0] = dados[0][q]
        tabela[h][1] = dados[x][q]
        h += 1
        m += 1
print(tabela)

anos = []
for dado in tabela:
    anos.append(dado[1])
valor = []
for dado in tabela:
    valor.append(dado[0])

barra = go.Bar(x=anos,
               y=valor,
               orientation='h',
               name='Tarifa Média [R$/MWh]',
               marker={'color': '#2178bb'})

config = go.Layout(title = dict(
                                text='Tarifa Média no Ano de 2012 [MWh/R$]',
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95),
                   yaxis={'title': 'região'},
                   xaxis={'title': ''})
trace = [barra]
fig11 = go.Figure(data=trace, layout=config)

fig11.show()

# -------------------------------------------------------------------------- 2013
#Acessa o arquivo
df = xlrd.open_workbook('202.xls')
tabela = df.sheet_by_name('Tabela 2.14')

# 2013
coluna = []
dados = []
x = '2013'
for i in range(8):
    for x in range(3):
        j = x + 1
        coluna = tabela.col_values(j)
        dados.append(coluna[10:15])
#print(dados)

# Relação com o ano 2013

h = 0
m = 0
tabela = [[1 for x in range(2)]for x in range(5)]
for q in range(5):
    for l in range(1):
        tabela[h][0] = dados[0][q]
        tabela[h][1] = dados[x][q]
        h += 1
        m += 1
print(tabela)

anos = []
for dado in tabela:
    anos.append(dado[1])
valor = []
for dado in tabela:
    valor.append(dado[0])

barra = go.Bar(x=anos,
               y=valor,
               orientation='h',
               name='Tarifa Média [R$/MWh]',
               marker={'color': '#2178bb'})

config = go.Layout(title = dict(
                                text='Tarifa Média no Ano de 2013 [MWh/R$]',
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95),
                   yaxis={'title': 'região'},
                   xaxis={'title': ''})
trace = [barra]
fig12 = go.Figure(data=trace, layout=config)

fig12.show()

# ------------------------------------------------------ 2014
df = xlrd.open_workbook('202.xls')
tabela = df.sheet_by_name('Tabela 2.14')

coluna = []
dados = []
x = '2014'
for i in range(8):
    for x in range(4):
        j = x + 1
        coluna = tabela.col_values(j)
        dados.append(coluna[10:15])
#print(dados)

# Relação com o ano 2014

h = 0
m = 0
tabela = [[1 for x in range(2)]for x in range(5)]
for q in range(5):
    for l in range(1):
        tabela[h][0] = dados[0][q]
        tabela[h][1] = dados[x][q]
        h += 1
        m += 1
print(tabela)

anos = []
for dado in tabela:
    anos.append(dado[1])
valor = []
for dado in tabela:
    valor.append(dado[0])

barra = go.Bar(x=anos,
               y=valor,
               orientation='h',
               name='Tarifa Média [R$/MWh]',
               marker={'color': '#2178bb'})

config = go.Layout(title = dict(
                                text='Tarifa Média no Ano de 2014 [MWh/R$]',
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95),
                   yaxis={'title': 'região'},
                   xaxis={'title': ''})
trace = [barra]
fig13 = go.Figure(data=trace, layout=config)

fig13.show()

# ------------------------------------------------------ 2015
df = xlrd.open_workbook('202.xls')
tabela = df.sheet_by_name('Tabela 2.14')

coluna = []
dados = []
x = '2015'
for i in range(8):
    for x in range(5):
        j = x + 1
        coluna = tabela.col_values(j)
        dados.append(coluna[10:15])
#print(dados)

# Relação com o ano 2015

h = 0
m = 0
tabela = [[1 for x in range(2)]for x in range(5)]
for q in range(5):
    for l in range(1):
        tabela[h][0] = dados[0][q]
        tabela[h][1] = dados[x][q]
        h += 1
        m += 1
print(tabela)

anos = []
for dado in tabela:
    anos.append(dado[1])
valor = []
for dado in tabela:
    valor.append(dado[0])

barra = go.Bar(x=anos,
               y=valor,
               orientation='h',
               name='Tarifa Média [R$/MWh]',
               marker={'color': '#2178bb'})

config = go.Layout(title = dict(
                                text='Tarifa Média no Ano de 2015 [MWh/R$]',
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95),
                   yaxis={'title': 'região'},
                   xaxis={'title': ''})
trace = [barra]
fig14 = go.Figure(data=trace, layout=config)

fig14.show()

# ----------------------------------------------------- 2016
df = xlrd.open_workbook('202.xls')
tabela = df.sheet_by_name('Tabela 2.14')

coluna = []
dados = []
x = '2016'
for i in range(8):
    for x in range(6):
        j = x + 1
        coluna = tabela.col_values(j)
        dados.append(coluna[10:15])
#print(dados)

# Relação com o ano 2016

h = 0
m = 0
tabela = [[1 for x in range(2)]for x in range(5)]
for q in range(5):
    for l in range(1):
        tabela[h][0] = dados[0][q]
        tabela[h][1] = dados[x][q]
        h += 1
        m += 1
print(tabela)

anos = []
for dado in tabela:
    anos.append(dado[1])
valor = []
for dado in tabela:
    valor.append(dado[0])

barra = go.Bar(x=anos,
               y=valor,
               orientation='h',
               name='Tarifa Média [R$/MWh]',
               marker={'color': '#2178bb'})

config = go.Layout(title = dict(
                                text='Tarifa Média no Ano de 2016 [MWh/R$]',
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95),
                   yaxis={'title': 'região'},
                   xaxis={'title': ''})
trace = [barra]
fig15 = go.Figure(data=trace, layout=config)

fig15.show()

# ----------------------------------------------------- 2017
df = xlrd.open_workbook('202.xls')
tabela = df.sheet_by_name('Tabela 2.14')

coluna = []
dados = []
x = '2017'
for i in range(8):
    for x in range(7):
        j = x + 1
        coluna = tabela.col_values(j)
        dados.append(coluna[10:15])
#print(dados)

# Relação com o ano 2017

h = 0
m = 0
tabela = [[1 for x in range(2)]for x in range(5)]
for q in range(5):
    for l in range(1):
        tabela[h][0] = dados[0][q]
        tabela[h][1] = dados[x][q]
        h += 1
        m += 1
print(tabela)

anos = []
for dado in tabela:
    anos.append(dado[1])
valor = []
for dado in tabela:
    valor.append(dado[0])

barra = go.Bar(x=anos,
               y=valor,
               orientation='h',
               name='Tarifa Média [R$/MWh]',
               marker={'color': '#2178bb'})

config = go.Layout(title = dict(
                                text='Tarifa Média no Ano de 2017 [MWh/R$]',
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95),
                   yaxis={'title': 'região'},
                   xaxis={'title': ''})
trace = [barra]
fig16 = go.Figure(data=trace, layout=config)

fig16.show()

# -------------------------------------------------------- 2018
df = xlrd.open_workbook('202.xls')
tabela = df.sheet_by_name('Tabela 2.14')

coluna = []
dados = []
x = '2018'
for i in range(8):
    for x in range(8):
        j = x + 1
        coluna = tabela.col_values(j)
        dados.append(coluna[10:15])
#print(dados)

# Relação com o ano 2018

h = 0
m = 0
tabela = [[1 for x in range(2)]for x in range(5)]
for q in range(5):
    for l in range(1):
        tabela[h][0] = dados[0][q]
        tabela[h][1] = dados[x][q]
        h += 1
        m += 1
print(tabela)

anos = []
for dado in tabela:
    anos.append(dado[1])
valor = []
for dado in tabela:
    valor.append(dado[0])

barra = go.Bar(x=anos,
               y=valor,
               orientation='h',
               name='Tarifa Média [R$/MWh]',
               marker={'color': '#2178bb'})

config = go.Layout(title = dict(
                                text='Tarifa Média no Ano de 2018 [MWh/R$]',
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95),
                   yaxis={'title': 'região'},
                   xaxis={'title': ''})
trace = [barra]
fig17 = go.Figure(data=trace, layout=config)

fig17.show()

[['Norte', 321.17], ['Nordeste', 297.09], ['Sudeste', 294.78], ['Sul', 277.22], ['Centro Oeste', 290.41]]


[['Norte', 276.68], ['Nordeste', 250.52], ['Sudeste', 260.24], ['Sul', 235.15], ['Centro Oeste', 257.74]]


[['Norte', 303.53], ['Nordeste', 269.07], ['Sudeste', 282.22], ['Sul', 264.28], ['Centro Oeste', 273.63]]


[['Norte', 373.04], ['Nordeste', 340.06], ['Sudeste', 413.05], ['Sul', 409.41], ['Centro Oeste', 398.08]]


[['Norte', 419.76], ['Nordeste', 367.44], ['Sudeste', 441.67], ['Sul', 415.39], ['Centro Oeste', 419.38]]


[['Norte', 477.74], ['Nordeste', 394.89], ['Sudeste', 431.77], ['Sul', 403.28], ['Centro Oeste', 426.95]]


[['Norte', 533.41], ['Nordeste', 450.99], ['Sudeste', 481.66], ['Sul', 456.43], ['Centro Oeste', 489.37]]


In [6]:
#---------------------------------------------------------Função de nomear---------------------------------------------------------------------------------------
def name_to_sigla(name):                                                                #Renomeia as siglas para relacionar as do arquivo json com as do arquivo csv
    if name == "Norte":                                                                 #Se sigla igual à Norte
        return "N"                                                                      #Retorna N
    if name == "Nordeste":                                                              #Se sigla igual à Nordeste
        return "NE"                                                                     #Retorna NE
    if name == "Centro-Oeste":                                                          #Se sigla igual à Centro-Oeste
        return "CO"                                                                     #Retorna CO
    if name == "Sudeste":                                                               #Se sigla igual à Sudeste
        return "SE"                                                                     #Retorna SE
    if name == "Sul":                                                                   #Se sigla igual à Sul
        return "S"                                                                      #Retorna S

In [7]:
#-------------------------------------------------Leitura de json-----------------------------------------------------------------
f = open("brazil_reg.json")                                                            #Abre o arquivo GEOJSON dividindo o mapa por região
br = json.loads(f.read())                                                              #Lê arquivo json

In [ ]:
#-------------------------------------------------Dash--------------------------------------------------------------

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])  
app.layout = html.Div(children=[                                                        #Div realiza a divisão da pagina 
    html.H1(children= 'Panorama da Energia Elétrica no Brasil', style = {"text-align":"center"}), 
    dcc.Dropdown(
        id = "filtro",
        options = [{'label': regiao, 'value': regiao} for regiao in regioes],
        value = "Norte",
        clearable = False
        ),
    dcc.Graph(id = "geracao"),
    
    html.Hr(),
    html.Div([dcc.Graph(figure = consumo_livre)]),
    
    html.Hr(), # Divisão por linha física
    html.Div([                                                                      # Divisão real entre o primeiro drop e os gráficos
        dcc.Dropdown(
            id='classe',
            options=[
                {'label': 'Norte', 'value': 'Nor'},
                {'label': 'Nordeste', 'value': 'Nord'},
                {'label': 'Sudeste', 'value': 'Sd'},
                {'label': 'Sul', 'value': 'Sl'},
                {'label': 'Centro-Oeste', 'value': 'CO'}],
            value='Nor'
            ),
        dcc.Graph(id='fig'), 
        
    html.Hr(),
    html.Div([
        dcc.Dropdown(
            id='demo-dropdown',
            options=[
                {'label': '2012', 'value': '2012'},
                {'label': '2013', 'value': '2013'},
                {'label': '2014', 'value': '2014'},
                {'label': '2015', 'value': '2015'},
                {'label': '2016', 'value': '2016'},
                {'label': '2017', 'value': '2017'},
                {'label': '2018', 'value': '2018'}],
            value='2012',
            ),
        dcc.Graph(id='fig11'),
        
    html.Hr(),                                                                          #Divisão de 1 linha
    html.Div([                                                                          #Divisão da pagina     
        dcc.Dropdown(id='classe1',                                                      #Nome da ID dos gráficos
        options=[
            {'label': '2012', 'value': '2012'},                                         #Declaração de label - 2012
            {'label': '2013', 'value': '2013'},                                         #Declaração de label - 2013
            {'label': '2014', 'value': '2014'},                                         #Declaração de label - 2014
            {'label': '2015', 'value': '2015'},                                         #Declaração de label - 2015
            {'label': '2016', 'value': '2016'},                                         #Declaração de label - 2016
            {'label': '2017', 'value': '2017'},                                         #Declaração de label - 2017
            {'label': '2018', 'value': '2018'}                                          #Declaração de label - 2018
        ],
        value='2012'),                                                                  #Inicia pela label 2012
                                                 
        ],style={'color':'blue', 'width':'20%'}),                                       #Color é referente a cor da grade
        dcc.Graph(id='fig_ano_2012'),
                                                 # Valor a princípio será de cama_mesa_banho    
    ],style={'color':'blue', 'width':'100%'}), # width= quanto de espaço toma da tela           
])])
@app.callback(                                                                          #Declaração de entradas e saída      
    Output('fig_ano_2012', 'figure'),                                                   #As saída são as figuras
    Input('classe1', 'value') 
)
#-------------------------------------------Gráfico-------------------------------------------------------------------------------------------------------------------------------
def luz_para_todos(year):
    f = open("Anuário Estatístico de Energia Elétrica 2020 - Workbook.xlsx - Tabela 2.24.csv")
    lines = f.read().split("\n")                                                           # f.read lê todo o conteúdo do arquivo e retorna uma string
                                                                                           # .split("\n") divide essa string do conteudo em linhas
    siglas = []                                                                            #Lista vazia que armazena as siglas

    populations = []                                                                       #Lista vazia que armazena os dados de população                                                                         #Determinação dos anos

    for l in lines[10:-2]:                                                                 #Lê as linhas começando da linha 10 e excluindo as duas últimas linhas
        ls = l.split(",")                                                                  #.split(",") separa elementos por vírgula da lista
        siglas.append(name_to_sigla(ls[1]))                                                #Insere na lista siglas cada elemento de name_to_sigla que esteja na posição 1 da lista
        population_str = ls[int(year[-1])].strip()                                         #.strip retira todos os espaços do começo e do inicio da string então '   -    ' fica '-'
        if population_str == '-':                                                          #Compara a string recebida com o '-'
            population = 0                                                                 #Se True '-' é igual a 0
        else:
            population = float(population_str)                                             #Se False transforma a string recebida num float

        populations.append(population)                                                     #Insere na lista population cada elemento de population

  
    d = {"regiao": siglas, "populacao": populations}  

    fig = px.choropleth(d, geojson=br, locations='regiao', color='populacao',             #Grafico de heatmap, com geojson dividido por região do br
                                 color_continuous_scale="PuBu",                           #Cor definida
                                featureidkey="properties.SIGLA",                          #Chave de interesse 
                               range_color=(0, 200),                                      #A internsidade das cores, o range
                               scope="south america",                                     #Mapa da america do sul
                               labels={'populacao':'População (mil)'},                    #Label do gráfico
                          )
    fig.update_layout(title = dict(
                                text='Distribuição Regional do Programa Luz Para Todos no Ano de {} [por mil habitantes]'.format(year),
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95),
    )
    return fig
                                                        


@app.callback(
   Output('fig11', 'figure'),
   Input('demo-dropdown', 'value')
)
def tarifa(x):
    if x == '2012':
        return fig11
    if x == '2013':
        return fig12
    if x == '2014':
        return fig13
    if x == '2015':
        return fig14
    if x == '2016':
        return fig15
    if x == '2017':
        return fig16
    if x == '2018':
        return fig17
    
@app.callback(
    Output('fig', 'figure'),
    Input('classe', 'value')
)
def gerargrafico(local):
    if local=='Nor':
        return fig 
    if local=='Nord':
        return fig1
    if local=='Sd':
        return fig2
    if local=='Sl':
        return fig3
    if local=='CO':
        return fig4
    
@app.callback(
    Output("geracao","figure"),
    Input("filtro","value")
)

def graficos(regiao):
    f = open("Anuário Estatístico de Energia Elétrica 2020 - Workbook.xlsx - Tabela 2.4.csv",encoding="utf8")
    # f.read lê todo o conteúdo do arquivo e retorna uma string
    # .split("\n") divide essa string do conteudo em linhas
    content = f.read().split("\n")
    
    #lista que armazena energia de cada região
    armazena_data = filtra_dados(regiao)

    #variável que armazena informações do gráfico      
    linha = go.Scatter(x = years, #anos do gráfico, eixo x
                       y = armazena_data, #energia do gráfico, eixo y
                       mode = 'lines', #modo do gráfico, tipo linhas
                       name = 'Geracao [GWh]', #nome das linhas
                       line = {'color': '#2178bb'}) #cor das linhas

    # abre o arquivo
    f = open("Anuário Estatístico de Energia Elétrica 2020 - Workbook.xlsx - Tabela 3.1.csv",encoding="utf8")
    # f.read lê todo o conteúdo do arquivo e retorna uma string
    # .split("\n") divide essa string do conteudo em linhas
    content = f.read().split("\n")
    
    #lista que armazena energia de cada região
    armazena_data = filtra_dados(regiao)
    #variável que armazena informações do gráfico
    barra = go.Bar(x = years,#anos do gráfico, eixo x
                       y = armazena_data, #energia do gráfico, eixo y
                       name = 'Consumo [GWh]',#nome das barras
                       marker = {'color': '#07325a'})#cor das barras


    config = go.Layout( title = dict(
                                text='Consumo Vs Geração na Região '+ regiao + ' [GWh]',
                                font=dict(size=20),
                                xref='paper', # Área central do gráfico
                                yref='container', # Área externa ao paper
                                x=0.5, # Faz o posicionamento horizontal do texto de acordo com o xref
                                y=0.95), # Faz o posicionamento vertical do texto de acordo com yref#título do gráfico
                                yaxis={'title':'Geração/Consumo [GWh]'}, #título eixo y
                                xaxis={'title':''}) #título eixo x
    trace = [linha, barra] #variável que armazena gráficos
    fig = go.Figure(data=trace, layout=config)
   
    return fig

if __name__ == '__main__':
    app.run_server(debug = True, use_reloader = False)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
